# Intro

This module is based on a Pluralsight course, Understanding and Applying Financial Risk Modeling Techniques (https://app.pluralsight.com/library/courses/financial-risk-modeling-techniques/table-of-contents), with this part Implementing Financial Risk Models in Python.

The code itself is nearly verbatim, but again, this is mostly for my own development, working through the peculiarities of Value at Risk (VaR) in both R and Python.

The general outline of this process is as follows:

- Load and clean Data
- Calculate returns
- Calculate historical variance
- Calculate systemic, idiosyncratic, and total variance
- Develop a range of stress variants, e.g. scenario-based possibilities
- Calculate VaR, as the worst case loss in a period for a particular probability

Note, The data required for this is in my Public Google Drive folder: https://drive.google.com/folderview?id=0B5Mi6_CV6da5aFc2Q0g3cHowVms&usp=sharing.

# Load and Clean Data

## Load Libraries

In [6]:

#changing directory
import os

#data frames
import pandas as pd

#statistcial work
import numpy as np

#for regresssion, used in factor models

import statsmodels.api as sm

#for calculating VaR
from scipy.stats import norm
import math


## Clean Data

In [7]:

# Get data
portfoliosFilePath = "Portfolios.csv"
portfolios = pd.read_csv(portfoliosFilePath, sep=",")
portfolios.head()

# format date
portfolios['AsOfDate'] = pd.to_datetime(portfolios['AsOfDate'], format='%m/%d/%Y', yearfirst = True)
portfolios.head()

# sort by date
portfolios = portfolios.sort_values(['AsOfDate'] , ascending=True)
portfolios.head()


,AsOfDate,AAPL,ADBE,CVX,GOOG,IBM,MDLZ,MSFT,NFLX,ORCL,SBUX,FVX,SP500
120,2007-01-03,11.107141,38.869999,50.777351,251.001007,79.242500,17.519524,24.118483,3.258571,15.696321,15.752188,4.819,1438.239990
119,2007-02-01,10.962033,39.250000,48.082939,224.949951,74.503204,16.019426,22.092464,3.218571,15.028588,13.930813,4.503,1406.819946
118,2007-03-01,12.037377,41.700001,51.900383,229.309311,75.561348,16.009354,21.857189,3.312857,16.583584,14.138198,4.537,1420.859985
117,2007-04-02,12.930043,41.560001,54.588032,235.925919,81.934280,16.924608,23.480597,3.167143,17.196436,13.984914,4.516,1482.369995
116,2007-05-01,15.701322,44.060001,57.598267,249.204208,85.786057,17.111704,24.146753,3.128572,17.726965,12.988567,4.853,1530.619995


# Estimating Historical Risk

## Calculating Returns

In [8]:

# calculate returns
returns = portfolios[[ key for key in dict(portfolios.dtypes) if dict(portfolios.dtypes)[key] in [ 'float64', 'int64']]].pct_change()
returns = returns[1:]
returns.head()

# add intercept column
returns['Intercept'] = 1
returns.head()

# create lists of names to easily filter returns
stockNames = list(returns)[0:10]
factorNames = list(returns)[10:13]

# set values for returns and weights
stockReturns = returns[stockNames]
factorReturns = returns[factorNames]
weights = np.array([1.0/len(stockNames)]*len(stockNames))

# cacluate portfolio variance
# Var(p) = W * COV(Y) * W_transposed
# matrix multiplication is done via np.dot
# first is transpose of weights
# second is the covariance of the returns
# third is the matrix of weights
historicalRisk = np.dot(np.dot(weights, stockReturns.cov()), weights.T)
historicalRisk


np.float64(0.0026512545422898237)

# Building Factor Models

In [9]:

factorData = factorReturns
factorData.head()

# calculate lm (regression) for FVX, SP500, and the stock value
# loops for each stock to create
# create data frame
modelCoeffs =[]
for stockName in stockNames:
    stockReturn = returns[stockName]
    model = sm.OLS(stockReturn, factorData)
    result = model.fit()
    modelCoeffRow = list(result.params)
    modelCoeffRow.append(np.std(result.resid,ddof=1))
    modelCoeffs.append(modelCoeffRow)
    #print(result.summary())
    
# cleanup data frame
modelCoeffs = pd.DataFrame(modelCoeffs)
modelCoeffs.columns = ["B_FVX", "B_SP500",  "Alpha", "ResidualVol"]
modelCoeffs["Names"] = stockNames
modelCoeffs


,B_FVX,B_SP500,Alpha,ResidualVol,Names
0,0.000026,1.199119,0.018665,0.075169,AAPL
1,0.029943,1.424392,0.005645,0.055747,ADBE
2,0.026186,0.836438,0.004126,0.045274,CVX
3,0.054624,1.019837,0.008247,0.068841,GOOG
4,0.045554,0.665858,0.004701,0.043510,IBM
5,-0.036069,0.672494,0.006146,0.046565,MDLZ
6,0.039786,0.988544,0.005911,0.056066,MSFT
7,-0.037955,0.829199,0.042232,0.171450,NFLX
8,0.031629,1.065534,0.004897,0.047754,ORCL
9,0.015355,1.087114,0.008556,0.065166,SBUX


# Factor Analysis - Idiosyncratic and Systemic Risk

Besides historical risk, one can decompose risk into systemic (beta) and idiosyncratic risk (alpha), the latte rbeing specific to the stock itself:

TotalVaR(P) = SystemicVaR(P) + IdisyncraticVaR(P)

## Systemic Variance

In [10]:

# SystemicVaR(P) = weights * betas * Cov(Factors) * betas_transposed * weights_transposed

# inner terms
factorCovariance = factorReturns[["SP500","FVX"]].cov()
reconstructedCov = np.dot(np.dot(modelCoeffs[["B_SP500", "B_FVX"]],factorCovariance),modelCoeffs[["B_SP500", "B_FVX"]].T)

# include outer terms
systemicVariance = np.dot(np.dot(weights, reconstructedCov), weights.T)
systemicVariance


np.float64(0.0019327420244913726)

## Idiosyncratic Variance

In [11]:

# idosyncraticVariance(P) = weights * var(residuals) * weight_transposed

idiosyncraticVariance = sum(weights * modelCoeffs["ResidualVol"] * weights * modelCoeffs["ResidualVol"])
idiosyncraticVariance


0.000586741277466821

## Total Variance

In [12]:

# totalVariance = systemicVariance + idiosyncraticVariance
factorBasedVariance = systemicVariance + idiosyncraticVariance
factorBasedVariance


np.float64(0.0025194833019581936)

# Scenario-based Stress Testing

## Creating Scenarios

In [13]:

# create a range of scenarios, stepping from min to max for factors, FVX and S&P500
fvxScenarios = np.arange(min(returns["FVX"]), max(returns["FVX"]), .05)
sp500Scenarios = np.arange(min(returns["SP500"]), max(returns["SP500"]), .02)

# build scenarios from fvxScenarios by sp500Scenarios
scenarios = []
for fvxValue in fvxScenarios:
    for sp500value in sp500Scenarios:
        scenario = [fvxValue, sp500value]
        for stockName in stockNames:
            alpha = float(modelCoeffs[modelCoeffs["Names"] == stockName]["Alpha"])
            beta_sp = float(modelCoeffs[modelCoeffs["Names"] == stockName]["B_SP500"])
            beta_fvx = float(modelCoeffs[modelCoeffs["Names"] == stockName]["B_FVX"])
            scenarioPredictedReturn = alpha + (beta_sp * sp500value) + (beta_fvx * fvxValue)
            scenario.append(scenarioPredictedReturn)
        scenarios.append(scenario)

scenarios = pd.DataFrame(scenarios)

# set column names
scenarios.columns = ["FVX","SP500","AAPL","ADBE","CVX","GOOG","IBM","MDLZ","MSFT","NFLX","ORCL","SBUX"]
scenarios.head()


C:\Users\igoej\AppData\Local\Temp\ipykernel_23684\2437322999.py:11: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha = float(modelCoeffs[modelCoeffs["Names"] == stockName]["Alpha"])
C:\Users\igoej\AppData\Local\Temp\ipykernel_23684\2437322999.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  beta_sp = float(modelCoeffs[modelCoeffs["Names"] == stockName]["B_SP500"])
C:\Users\igoej\AppData\Local\Temp\ipykernel_23684\2437322999.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  beta_fvx = float(modelCoeffs[modelCoeffs["Names"] == stockName]["B_FVX"])


,FVX,SP500,AAPL,ADBE,CVX,GOOG,IBM,MDLZ,MSFT,NFLX,ORCL,SBUX
0,-0.310883,-0.169425,-0.184503,-0.244991,-0.145728,-0.181520,-0.122273,-0.096577,-0.173942,-0.086455,-0.185463,-0.180402
1,-0.310883,-0.149425,-0.160521,-0.216503,-0.128999,-0.161123,-0.108956,-0.083128,-0.154171,-0.069871,-0.164152,-0.158659
2,-0.310883,-0.129425,-0.136538,-0.188015,-0.112270,-0.140726,-0.095639,-0.069678,-0.134400,-0.053287,-0.142842,-0.136917
3,-0.310883,-0.109425,-0.112556,-0.159527,-0.095542,-0.120329,-0.082322,-0.056228,-0.114629,-0.036703,-0.121531,-0.115175
4,-0.310883,-0.089425,-0.088574,-0.131039,-0.078813,-0.099933,-0.069004,-0.042778,-0.094858,-0.020119,-0.100220,-0.093432


## Calculating Variance

In [14]:

# totalVariance(P) = weights * cov(scenarios) * weight_transposed
scenariosCov = scenarios[stockNames].cov()
scenarioTotalVariance = np.dot(np.dot(weights, scenariosCov), weights.T)
scenarioTotalVariance


np.float64(0.006272120969460552)

# Quantifying the Worst Case

In [15]:

# VaR = P x Z_alpha x stdDev
def calculateVaR(risk, confidenceLevel, principal = 1, numMonths = 1):
    vol = math.sqrt(risk)
    return abs(principal * norm.ppf(1-confidenceLevel, 0, 1) * vol * math.sqrt(numMonths))


In [16]:

# Worst Case: VaR based on scenario-based stress testing
calculateVaR(scenarioTotalVariance, 0.99)


np.float64(0.1842391287037656)

In [17]:

# Allternaitve Risk Measure:  VaR based on factors
calculateVaR(factorBasedVariance, 0.99)


np.float64(0.11676976342787522)

In [18]:

# Allternaitve Risk Measure:  VaR based on history
calculateVaR(historicalRisk, 0.99)


np.float64(0.11978443009308773)